In [25]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="5" color="red" >ch09_파일 입출력 프로그래밍</font></b>

- 1. *txt
    김회원, 33, 아무동9
    이회원, 22, 무시기동8
- 2. pickle파일(*.data): 객체리스트 저장, (머신러닝모델저장용)
- 3. *.csv
    name,age,address
    "김회원",33,"아무동9"
    "이회원",22,"무시기동8"
- 4. *.json
    [{'name':'김회원','age':33,'address':'아무동9'},
    {'name':'이회원','age':22,'address':'무시기동8'}]
- 5. *.h5 이나 *.hdf5 (=>딥러닝 모델 저장)
    HDVview 로 읽을 수 있음

- 6. *xml

<customer>
	<name>김회원</name>
	<age>33</age>
	<address>아무동9</address>
</customer>
<customer>
	<name>이회원</name>
	<age>22</age>
	<address>무시기동9</address>
</customer>

# 1절. txt파일에 데이터 저장하고 불러오기

In [8]:
f = open('data/ch09.txt', 'w')
print('쓰기가능한지 여부: ', f.writable())

쓰기가능한지 여부:  True


In [9]:
f.write('Hello\nWorld')
f.close() # close() 쓰기한 내용이 저장이 X

In [10]:
# mode : 'r' 또는 'rt' : text 읽기 모드
       # 'rb' : 바이너리 읽기 모드
       # 'w' 또는 'wt' : text 쓰기 모드(파일이 있으면 덮어씀, 파일이 없으면 파일 생성)
       # 'wb' : 바이너리쓰기 모드
       # 'a' 또는 'at' : text 추가 모드(파일이 있으면 append, 파일이 없으면 파일 생성)
       # 읽기모드에서는 파일이 없으면 예외/쓰기 모드에서는 폴더가 없으면 예외
# encoding
    # euc-kr (한글완성형) 믜x
    # cp949 (확장된 한글완성형) : open()함수 기본 encoding 방식(win)
    # utf-8 (한글조합형): open()함수 기본 encoding방식(mac, linux)
with open('data/ch09.txt', 'w') as f:
    print('쓰기 가능한지 여부 :', f.writable())
    f.write('Hello\nPython')

쓰기 가능한지 여부 : True


In [11]:
with open('data/ch09.txt', 'a', encoding='utf-8') as f:
    # 1방법
    f.write('홍길동, 33, 아무동9\n')
    f.write('맹길동, 33, 아무동9\n')
    # 2방법
    textlist = ['홍길동, 33, 아무동9\n', '김길동, 33, 아무동9\n']
    for line in textlist:
        f.write(line)
    # 3방법
    f.writelines(textlist)

In [14]:
# 한줄씩 읽기
with open('data/ch09.txt', 'r', encoding='utf-8') as f:
    line = f.readline()
    while line!='':
        print(line,end='')
        line = f.readline()

홍길동, 33, 아무동9
맹길동, 33, 아무동9
홍길동, 33, 아무동9
김길동, 33, 아무동9
홍길동, 33, 아무동9
김길동, 33, 아무동9


In [17]:
# 모든 줄을 읽기
with open('data/ch09.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    print(lines)

['홍길동, 33, 아무동9\n', '맹길동, 33, 아무동9\n', '홍길동, 33, 아무동9\n', '김길동, 33, 아무동9\n', '홍길동, 33, 아무동9\n', '김길동, 33, 아무동9\n']


In [18]:
# 모두 읽기
with open('data/ch09.txt', 'r', encoding='utf-8') as f:
    lines = f.read()
    print(lines)

홍길동, 33, 아무동9
맹길동, 33, 아무동9
홍길동, 33, 아무동9
김길동, 33, 아무동9
홍길동, 33, 아무동9
김길동, 33, 아무동9



# 2절. 피클을 이용한 객체 저장 및 불러오기
## 2.1 형식이 있는 txt 데이터 불러오기

In [1]:
class Member:
    def __init__(self, name, age, email, address):
        self.name = name
        self.age = age
        self.email = email
        self.address = address
    def __str__(self):
        return "{}, {}, {}, {}".format(self.name,
                                      "성년" if self.age>18 else "미성년",
                                      self.email,
                                      self.address)
    def as_dict(self):
        return {
            'name':self.name,
            'age':self.age,
            'email':self.email,
            'address':self.address
                }
    def __eq__(self, other):
        return self.name == other.name and \
                self.age==other.age and \
                self.email==other.email and \
                self.address==other.address

In [2]:
user1 = Member('홍',20,'abc@abc.com','신림동')
user2 = Member('홍',20,'abc@abc.com','신림동')
print(user1==user2)
print(user1.__eq__(user2)) # 같은 내용이면 True

True
True


In [4]:
# 형식이 있는 txt 파일 내용을 member list(피클 저장), 딕셔너리 list(데이터프레임) 로 저장
user_list = [] # member list
user_dict = [] # 딕셔너리 list
with open('data/ch09_member.txt', 'r', encoding='utf-8') as txt_file:
    lines = txt_file.readlines()
# print(lines)
for line in lines:
    data = line.split(',')
#     print(data)
    name = data[0]
    age = int(data[1].strip()) # .strip() : 좌우 공백(space, \t, \n) 제거
    email = data[2].strip()
    address = data[3].strip()
    user = Member(name, age, email, address)
    user_list.append(user)
    user_dict.append(user.as_dict())
#     user_dict.append(user.__dict__)

In [5]:
user_dict

[{'name': '홍길동', 'age': 20, 'email': 'kildong@hong.com', 'address': '서울시 관악구'},
 {'name': '홍길숙',
  'age': 25,
  'email': 'kilsuk1@hong.com',
  'address': '서울시 영등포구'},
 {'name': '신길동', 'age': 30, 'email': 'shinkil@hong.com', 'address': '서울시 동작구'}]

In [6]:
for user in user_list:
    print(user)

홍길동, 성년, kildong@hong.com, 서울시 관악구
홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
신길동, 성년, shinkil@hong.com, 서울시 동작구


## 2.2 피클링
- 객체 리스트(user_list) -> 피클파일로 쓰기
- 피클 파일을 읽기 -> 객체 리스트(load_user_list)

In [7]:
# 피클링을 이용한 객체를 저장하기
import pickle
with open('data/ch09_member.data', 'wb') as f:
    pickle.dump(user_list, f)

In [8]:
# 피클링을 이용한 파일에서 객체 데이터로 읽기
with open('data/ch09_member.data', 'rb') as f:
    load_user_list = pickle.load(f)

In [9]:
user_list == load_user_list

True

In [12]:
for idx in range(len(user_list)):
    print(idx, user[idx])
    print(idx, load_user_list[idx])

0 홍길동, 성년, kildong@hong.com, 서울시 관악구
0 홍길동, 성년, kildong@hong.com, 서울시 관악구
1 홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
1 홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
2 신길동, 성년, shinkil@hong.com, 서울시 동작구
2 신길동, 성년, shinkil@hong.com, 서울시 동작구


In [26]:
for idx, (user, load_user) in enumerate(zip(user_list, load_user_list)):
    print(idx, user)
    print(idx, load_user)
    print(user==load_user)

0 홍길동, 성년, kildong@hong.com, 서울시 관악구
0 홍길동, 성년, kildong@hong.com, 서울시 관악구
True
1 홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
1 홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
True
2 신길동, 성년, shinkil@hong.com, 서울시 동작구
2 신길동, 성년, shinkil@hong.com, 서울시 동작구
True


In [27]:
result = []
for user, load_user in enumerate(zip(user_list, load_user_list)):
    result.append(user==load_user)
all(result)

False

# 3절. CSV형식 파일 읽기/쓰기
- CSV 파일 <-> 리스트데이터 3.1 / 3.2
- CSV 파일 <-> 딕셔너리데이터 3.3 / 3.4

## 3.1 reader

In [31]:
import csv
with open('data/ch09_member1.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    result = list(reader)
print(result)

[['홍길동', '20', 'kildong@hong.com', '서울시 관악구'], ['김길동', '40', 'kimdong@hong.com', '서울시 영등포구'], ['신길동', '30', 'sindong@hong.com', '서울시 동작구']]


In [30]:
# ""(따옴표)가 없는 데이터는 numeric으로 
import csv
with open('data/ch09_member1.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f,
                       quoting=csv.QUOTE_NONNUMERIC)
    result = list(reader)
print(result)

[['홍길동', 20.0, 'kildong@hong.com', '서울시 관악구'], ['김길동', 40.0, 'kimdong@hong.com', '서울시 영등포구'], ['신길동', 30.0, 'sindong@hong.com', '서울시 동작구']]


In [34]:
import csv
with open('data/ch09_member1.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f,
                       quoting=csv.QUOTE_NONNUMERIC)
    result = list(reader)
dict_list = []
for data in result:
    dict_list.append({
        'name':data[0],
        'age':int(data[1]),
        'email':data[2],
        'address':data[3]
    })
print(dict_list)

[{'name': '홍길동', 'age': 20, 'email': 'kildong@hong.com', 'address': '서울시 관악구'}, {'name': '김길동', 'age': 40, 'email': 'kimdong@hong.com', 'address': '서울시 영등포구'}, {'name': '신길동', 'age': 30, 'email': 'sindong@hong.com', 'address': '서울시 동작구'}]


## 3.2 writer

In [36]:
user_list = [['홍길동', '20', 'kildong@hong.com', '서울시 관악구'],
             ['김길동', '40', 'kimdong@hong.com', '서울시 영등포구']]

In [37]:
with open('data/ch09_member1_write.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(user_list)

In [38]:
with open('data/ch09_member1_write.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f, quoting=csv.QUOTE_NONNUMERIC)
    writer.writerows(user_list)

## 3.3 DictReader

In [45]:
import csv
with open('data/ch09_member3.csv', 'r', encoding='utf-8') as f:
    dict_reader = csv.DictReader(f)
    dict_list = list(dict_reader)
print(dict_list)
    
for row in dict_list:
    if row['job'] is None:
        print(row['name'], row['age'], row['email'], row['address'])
    else:
        print(row['name'], row['age'], row['email'], row['address'], row['job'])

[{'name': '홍길동', 'age': '20', 'email': 'h@h.com', 'address': '서울시 관악구', 'job': None}, {'name': '신길동', 'age': '40', 'email': 's@h.com', 'address': '서울시 영등포구', 'job': '팀장'}, {'name': '김길동', 'age': '30', 'email': 'k@h.com', 'address': '서울시 동작구', 'job': None}]
홍길동 20 h@h.com 서울시 관악구
신길동 40 s@h.com 서울시 영등포구 팀장
김길동 30 k@h.com 서울시 동작구


In [53]:
# header가 없는 csv파일(ch09_member1-cp.csv)을 딕셔너리로 읽기
with open('data/ch09_member1-cp.csv', encoding='cp949') as f:
    dict_reader = csv.DictReader(f,
                                 fieldnames=['Name','Age','Email','Address','Job'])
#     print(list(dict_reader))
    for row in dict_reader:
        print(1)
        print(row)

{'Name': '홍길동', 'Age': '20', 'Email': 'kildong@hong.com', 'Address': '서울시 관악구', 'Job': ''}
{'Name': '김길동', 'Age': '40', 'Email': 'kimdong@hong.com', 'Address': '서울시 영등포구', 'Job': '팀장'}
{'Name': '신길동', 'Age': '30', 'Email': 'sindong@hong.com', 'Address': '서울시 동작구', 'Job': ''}


In [57]:
# header가 없는 csv파일(ch09_member1-cp.csv)을 딕셔너리로 읽기
with open('data/ch09_member1-cp.csv', encoding='cp949') as f:
    dict_reader = csv.DictReader(f,
                                 fieldnames=['Name','Age','Email','Address'],
                                restkey='Job')
    result = list(dict_reader)
#     print(list(dict_reader))
for row in result:
    print(row)

{'Name': '홍길동', 'Age': '20', 'Email': 'kildong@hong.com', 'Address': '서울시 관악구', 'Job': ['']}
{'Name': '김길동', 'Age': '40', 'Email': 'kimdong@hong.com', 'Address': '서울시 영등포구', 'Job': ['팀장']}
{'Name': '신길동', 'Age': '30', 'Email': 'sindong@hong.com', 'Address': '서울시 동작구', 'Job': ['']}


## 3.4 DictWriter
- 딕셔너리 리스트 -> CSV파일

In [62]:
user1 = {'name':'홍길동','age':22, 'email':'a@a.com', 'address':'신림동'}
user2 = {'name':'신길동','age':32, 'email':'b@a.com', 'address':'신림동'}
user3 = {'name':'김길동','age':42, 'email':'c@a.com', 'address':'신림동'}
user_list = [user1, user2, user3]
fieldnames = list(user1.keys())


In [66]:
with open('data/ch09_member4.csv', 'w', encoding='utf-8', newline='') as f:
    dict_writer = csv.DictWriter(f,
                                fieldnames=fieldnames)
    dict_writer.writeheader() # header 쓰기
    dict_writer.writerows(user_list)

## CSV <-> 데이터프레임

In [5]:
import pandas as pd
member = pd.read_csv('data/ch09_member3.csv')
member

,name,age,email,address,job
0,홍길동,20,h@h.com,서울시 관악구,NaN
1,신길동,40,s@h.com,서울시 영등포구,팀장
2,김길동,30,k@h.com,서울시 동작구,NaN


In [6]:
type(member)

pandas.core.frame.DataFrame

# 4절. JSON 데이터 저장하고 불러오기(dump, load)
- 딕셔너리리스트 <-> JSON 파일(기본)
- 객체리스트 <-> JSON 파일
## 4.1 dump(파일 출력)

In [7]:
data = [{'name': '홍길동', 'age': 20, 'email': 'kildong@hong.com', 'address': '서울'},
        {'name': '김길동', 'age': 30, 'email': 'kimdong@hong.com', 'address': '인천'}]

In [6]:
# ensure_ascii 매개변수
    #True : 비 ASCII 문은 유니코드 형태로 저장
    #False : 비 ASCII 문자 원래 형태로 저장
import json
with open('data/ch09_member1.json', 'w', encoding='utf-8') as jsonfile:
    json.dump(data, #딕셔너리 리스트
             jsonfile,
             ensure_ascii=False)

NameError: name 'data' is not defined

In [17]:
class Member:
    def __init__(self, name, age, email, address):
        self.name = name
        self.age = age
        self.email = email
        self.address = address
    def __str__(self):
        return "{}, {}, {}, {}".format(self.name,
                                      self.age,
                                      self.email,
                                      self.address)
    def as_dict(self):
        return {
            'name':self.name,
            'age':self.age,
            'email':self.email,
            'address':self.address
                }
    def __eq__(self, other):
        if isinstance(other, Member):
            return self.__dict__ == other.__dict__
#             return self.__str__() == other.__str__()
        
        else:
            return False

In [18]:
user1 = Member('홍길동',22,'a@a.com','신림동')
user2 = Member('홍길동',22,'a@a.com','신림동')
print(user1.__eq__('홍길동'))
print(user1=='홍길동')

False
False


In [19]:
member_list = [Member('홍길동',22,'a@a.com','서울'),
               Member('신길동',32,'a@a.com','서울'),
               Member('김길동',42,'a@a.com','서울')]

In [20]:
with open('data/ch09_member1.json', 'w', encoding='utf-8') as jsonfile:
    json.dump(member_list, # 객체 리스트
             jsonfile,
             ensure_ascii=False,
             indent='\t',
             default=Member.as_dict # 객체를 딕셔너리로 return 하는 인스턴스 함수
             )

## 4.2 load(파일 입력)
- json 파일 -> 딕셔너리 리스트(기본)
                  ↓
- json 파일 -> 객체리스트

In [21]:
def as_member(dic):
    '매개변수로 딕셔너리를 받아 Member 객체를 return'
    return Member(dic.get('name'), dic['age'], dic.get('email'), dic.get('address'))

In [22]:
member = as_member({'name':'김길동', 'age':20, 'email':'z@a.com', 'address':'관약'})
print(member)

김길동, 20, z@a.com, 관약


In [23]:
with open('data/ch09_member.json', 'r', encoding='utf-8') as f:
    load_data = json.load(f)
load_data

[{'name': '홍길동', 'age': 20, 'email': 'kildong@hong.com', 'address': '서울'},
 {'name': '김길동', 'age': 30, 'email': 'kimdong@hong.com', 'address': '인천'}]

In [24]:
with open('data/ch09_member1.json', 'r', encoding='utf-8') as f:
    load_member_list = json.load(f, object_hook=as_member)
for load_member in load_member_list:
    print(load_member)

홍길동, 22, a@a.com, 서울
신길동, 32, a@a.com, 서울
김길동, 42, a@a.com, 서울


## JSON -> 데이터프레임

In [16]:
import pandas as pd
pd.read_json('data/ch09_member1.json')
# pd.함수() : encoding='utf-8' 기본값
# open(파일, 모드)함수 : encoding='cp949' 기본값

,name,age,email,address
0,홍길동,22,a@a.com,서울
1,신길동,32,a@a.com,서울
2,김길동,42,a@a.com,서울


# 5절. 연습문제
## 실습형

In [26]:
# 1.
if __name__ == "__main__":
    main()

NameError: name 'main' is not defined